In [1]:
%matplotlib inline
import os
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

/home/sam/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:53: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


## Importation et merge des premiers dataset

**airports_kaggle** : 
    - code IATA
    - l'abréviation de la ville 
    - le nom de la ville 
    - Latitude/Longitude
    
**States** : permet d'avoir le nom complet de la ville

**airports_openflights** : 
    - Altitude de l'aréoport
    - TimeZone 
Une fois mergé, on ne garde que les 20 aéroports qui nous interesse.

In [2]:
# https://www.kaggle.com/usdot/flight-delays#airports.csv
airports_kaggle = pd.read_csv(
    os.path.join('data', 'airports_kaggle.csv')
).rename(columns={"IATA_CODE":"IATA"}).drop(columns=['AIRPORT','COUNTRY'])

# CSV des abréviations des states : "Alabama":"AL"
states = pd.read_csv(
    os.path.join('data', 'states.csv')
).rename(columns={"State":"STATE_FULL","Abbreviation":"STATE" })

airports_kaggle = airports_kaggle.merge(states, on="STATE")

# https://openflights.org/data.html
airports_openflights = pd.read_csv(
    os.path.join('data', 'airports.csv'), index_col="Airport_ID"
)[['IATA', 'Altitude', "Tz"]]
airports_merged = airports_kaggle.merge(airports_openflights, on ="IATA").drop(columns=['STATE'])

In [3]:
# On ne garde que les airports du starting kit
air_passenger_airports = ['ATL', 'BOS', 'CLT', 'DEN', 'DFW', 'DTW', 'EWR', 'IAH', 'JFK',
       'LAS', 'LAX', 'LGA', 'MCO', 'MIA', 'MSP', 'ORD', 'PHL', 'PHX',
       'SEA', 'SFO']
airports = airports_merged[airports_merged['IATA'].isin(air_passenger_airports)]
print( sorted(airports.IATA.values) == sorted(air_passenger_airports))
airports['Tz'] = airports['Tz'].apply(lambda row : row.split("/")[1])
airports.sort_values(by="STATE_FULL").head()

True


/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,IATA,CITY,LATITUDE,LONGITUDE,STATE_FULL,Altitude,Tz
285,PHX,Phoenix,33.43417,-112.00806,Arizona,1135,Phoenix
57,LAX,Los Angeles,33.94254,-118.40807,California,125,Los_Angeles
68,SFO,San Francisco,37.61900,-122.37484,California,13,Los_Angeles
138,DEN,Denver,39.85841,-104.66700,Colorado,5431,Denver
262,MIA,Miami,25.79325,-80.29056,Florida,8,New_York


## Population 
Nous avons les populations depuis 2010 jusqu'à 2018. 

Pour ne pas 'tricher' nous avons fait le choix de ne conserver que l'estimiation faite en Juillet 2011.

In [4]:
# https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-cities-and-towns.html
us_population = pd.read_csv(
    os.path.join('data', 'us_population.csv'),encoding='latin-1'
)
us_population = us_population[['Geography','Population Estimate (as of July 1) - 2011']].rename(columns={"Population Estimate (as of July 1) - 2011":"population_2011"})
print(us_population.shape)
# Decomposition de la variables Geography
us_population['STATE_FULL'] = us_population.Geography.apply(lambda geo : geo.split(", ")[1])

# on ne garde que les états
us_population = us_population.drop(columns='Geography').groupby("STATE_FULL").mean().reset_index()
us_population.population_2011 = us_population.population_2011.astype(int)
us_population.head()

(19495, 2)


,STATE_FULL,population_2011
0,Alabama,6319
1,Alaska,3348
2,Arizona,56061
3,Arkansas,3818
4,California,64980


In [9]:
# Ici on merge les IATA des airpots et la population grace aux noms d'état
#iata_pop = airports[airports.STATE_FULL.isin(airports.STATE_FULL.unique())][['IATA','STATE_FULL']].rename(columns={'STATE_FULL':'State'})
airports_extended = airports.merge(us_population, on="STATE_FULL")
airports_extended = airports_extended.drop(columns=["CITY"])#.rename(columns={"Population Estimate (as of July 1) - 2011":"population_2011"})
airports_extended

,IATA,LATITUDE,LONGITUDE,STATE_FULL,Altitude,Tz,population_2011
0,PHL,39.87195,-75.24114,Pennsylvania,36,New_York,5543
1,DFW,32.89595,-97.03720,Texas,607,Chicago,15567
2,IAH,29.98047,-95.33972,Texas,97,Chicago,15567
3,ATL,33.64044,-84.42694,Georgia,1026,New_York,8082
4,BOS,42.36435,-71.00518,Massachusetts,20,New_York,63593
5,LAX,33.94254,-118.40807,California,125,Los_Angeles,64980
6,SFO,37.61900,-122.37484,California,13,Los_Angeles,64980
7,EWR,40.69250,-74.16866,New Jersey,18,New_York,12957
8,JFK,40.63975,-73.77893,New York,13,New_York,20688
9,LGA,40.77724,-73.87261,New York,21,New_York,20688


In [8]:
airports_extended[['IATA','STATE_FULL']].to_dict()

{'IATA': {0: 'PHL',
  1: 'DFW',
  2: 'IAH',
  3: 'ATL',
  4: 'BOS',
  5: 'LAX',
  6: 'SFO',
  7: 'EWR',
  8: 'JFK',
  9: 'LGA',
  10: 'DTW',
  11: 'DEN',
  12: 'CLT',
  13: 'MSP',
  14: 'SEA',
  15: 'ORD',
  16: 'MCO',
  17: 'MIA',
  18: 'LAS',
  19: 'PHX'},
 'STATE_FULL': {0: 'Pennsylvania',
  1: 'Texas',
  2: 'Texas',
  3: 'Georgia',
  4: 'Massachusetts',
  5: 'California',
  6: 'California',
  7: 'New Jersey',
  8: 'New York',
  9: 'New York',
  10: 'Michigan',
  11: 'Colorado',
  12: 'North Carolina',
  13: 'Minnesota',
  14: 'Washington',
  15: 'Illinois',
  16: 'Florida',
  17: 'Florida',
  18: 'Nevada',
  19: 'Arizona'}}

In [35]:
airports_extended = airports_extended.rename(columns={
    'IATA':'iata',
    'LATITUDE':'lat',
    'LONGITUDE':'long',
    'STATE_FULL':'state',
    'Altitude':'altitude',
    'Tz':'tz',
    'population_2011':'pop_2011'
})
#airports_extended.set_index('iata').to_dict('index')
mon_dico = airports_extended.set_index('iata').to_dict('index')
mon_dico['PHL']['lat']

SyntaxError: invalid syntax (<ipython-input-35-2640712deb16>, line 12)

## Merging with external_data

In [9]:
external_data = pd.read_csv('submissions/submit/external_data.csv', 
                            index_col=0)
external_data

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Holiday,year,month,TOTAL_DEPARTURE,TOTAL_DESTINATION
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,0,2011,9,3582088,3580851
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,0,2011,9,3582088,3580851
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,0,2011,9,3582088,3580851
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,0,2011,9,3582088,3580851
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,2011,9,3582088,3580851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,0,2013,3,1164319,1141295
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,0,2013,3,1164319,1141295
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,0,2013,3,1164319,1141295
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,0,2013,3,1164319,1141295
